In [25]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms


# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper-parameters 
input_size = 784
hidden_size = 500
num_classes = 10
num_epochs = 20
batch_size = 128
learning_rate = 0.005

# MNIST dataset 
train_dataset = torchvision.datasets.KMNIST(root='../../data',        #instead of KMNIST , just type CIFAR10 . Please check the spelling
                                           train=True, 
                                           transform=transforms.ToTensor(),  
                                           download=True)

test_dataset = torchvision.datasets.KMNIST(root='../../data', 
                                          train=False, 
                                          transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

# Fully connected neural network with one hidden layer
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size) 
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)  
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

model = NeuralNet(input_size, hidden_size, num_classes).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)  

# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):  
        # Move tensors to the configured device
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 300 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

# # Test the model
# # In test phase, we don't need to compute gradients (for memory efficiency)

Epoch [1/20], Step [300/469], Loss: 0.1445
Epoch [2/20], Step [300/469], Loss: 0.0911
Epoch [3/20], Step [300/469], Loss: 0.0828
Epoch [4/20], Step [300/469], Loss: 0.0885
Epoch [5/20], Step [300/469], Loss: 0.0326
Epoch [6/20], Step [300/469], Loss: 0.0721
Epoch [7/20], Step [300/469], Loss: 0.0343
Epoch [8/20], Step [300/469], Loss: 0.0304
Epoch [9/20], Step [300/469], Loss: 0.0385
Epoch [10/20], Step [300/469], Loss: 0.0084
Epoch [11/20], Step [300/469], Loss: 0.0369
Epoch [12/20], Step [300/469], Loss: 0.0162
Epoch [13/20], Step [300/469], Loss: 0.0312
Epoch [14/20], Step [300/469], Loss: 0.0553
Epoch [15/20], Step [300/469], Loss: 0.0244
Epoch [16/20], Step [300/469], Loss: 0.0316
Epoch [17/20], Step [300/469], Loss: 0.0045
Epoch [18/20], Step [300/469], Loss: 0.0060
Epoch [19/20], Step [300/469], Loss: 0.1502
Epoch [20/20], Step [300/469], Loss: 0.1290


In [26]:
testing=[]
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        testing.append((images, labels))
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test accuracy: {} %'.format(100 * correct / total))
    correct = 0
    total = 0
    for images, labels in train_loader:
        testing.append((images, labels))
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Train accuracy: {} %'.format(100 * correct / total))

Test accuracy: 90.97 %
Train accuracy: 99.45833333333333 %


In [27]:
for param in model.parameters():
    param.requires_grad= False

Now generating adverserial examples for the model

In [28]:
import numpy as np
l =[]
sign = []
for epoch in range(1):
    for i, (images, labels) in enumerate(train_loader):  
# Move tensors to the configured device
        images = images.reshape(-1,28*28).to(device)
        images.requires_grad=True
        labels = labels.to(device)
        # Forward pass
        outputs = model(images)
   #     print(labels.shape)
        loss = criterion(outputs, labels)
        loss.backward()
        c =images.grad.sign()
        images = images+ 0.01* c
        sign.append(c)
        l.append((images,labels))



In [30]:
import numpy as np
l_test =[]
sign_test = []
for epoch in range(1):
    for i, (images, labels) in enumerate(test_loader):  
# Move tensors to the configured device
        images = images.reshape(-1,28*28).to(device)
        images.requires_grad=True
        labels = labels.to(device)
        # Forward pass
        outputs = model(images)
   #     print(labels.shape)
        loss = criterion(outputs, labels)
        loss.backward()
        c =images.grad.sign()
        images = images+ 0.01* c
        sign_test.append(c)
        l_test.append((images,labels))



In [29]:
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in l:
        images = images.reshape(-1,28*28).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
   #     print(predicted)
        correct += (predicted == labels).sum().item()

    print('Accuracy on adverserial images of training dataa: {} %'.format(100 * correct / total))

Accuracy on adverserial images of training dataa: 96.34 %


In [31]:
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in l_test:
        images = images.reshape(-1,28*28).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
   #     print(predicted)
        correct += (predicted == labels).sum().item()

    print('Accuracy on adverserial images of testing data: {} %'.format(100 * correct / total))

Accuracy on adverserial images of testing data: 82.97 %


**Now doing adverserial training**

In [32]:
model_new = NeuralNet(input_size, hidden_size, num_classes).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model_new.parameters(), lr=learning_rate)  

In [33]:
tot = total_step
total_step = 2*len(train_loader)

for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):  
        # Move tensors to the configured device
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
    
        # Forward pass
        outputs = model_new(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if (i+1) % 300 == 0:
                print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                       .format(epoch+1, num_epochs, i+1, total_step, loss.item()))
    for (images, labels) in l:
        i+=1
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model_new(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
     #   print(i)
        if (i+1) % 300 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/20], Step [300/938], Loss: 0.2502
Epoch [1/20], Step [600/938], Loss: 0.0726
Epoch [1/20], Step [900/938], Loss: 0.3586
Epoch [2/20], Step [300/938], Loss: 0.0943
Epoch [2/20], Step [600/938], Loss: 0.0251
Epoch [2/20], Step [900/938], Loss: 0.2766
Epoch [3/20], Step [300/938], Loss: 0.0136
Epoch [3/20], Step [600/938], Loss: 0.0488
Epoch [3/20], Step [900/938], Loss: 0.1411
Epoch [4/20], Step [300/938], Loss: 0.0671
Epoch [4/20], Step [600/938], Loss: 0.0870
Epoch [4/20], Step [900/938], Loss: 0.0754
Epoch [5/20], Step [300/938], Loss: 0.0136
Epoch [5/20], Step [600/938], Loss: 0.0166
Epoch [5/20], Step [900/938], Loss: 0.0158
Epoch [6/20], Step [300/938], Loss: 0.0009
Epoch [6/20], Step [600/938], Loss: 0.0082
Epoch [6/20], Step [900/938], Loss: 0.0142
Epoch [7/20], Step [300/938], Loss: 0.0725
Epoch [7/20], Step [600/938], Loss: 0.0086
Epoch [7/20], Step [900/938], Loss: 0.0835
Epoch [8/20], Step [300/938], Loss: 0.0401
Epoch [8/20], Step [600/938], Loss: 0.0317
Epoch [8/20

In [35]:
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in l_test:
        images = images.reshape(-1,28*28).to(device)
        labels = labels.to(device)
        outputs = model_new(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy on adverserial images of test data: {} %'.format(100 * correct / total))

Accuracy on adverserial images of test data: 89.37 %


In [36]:
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in l:
        images = images.reshape(-1,28*28).to(device)
        labels = labels.to(device)
        outputs = model_new(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy on adverserial images of train data: {} %'.format(100 * correct / total))

Accuracy on adverserial images of train data: 99.55 %
